In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO/

/content/drive/MyDrive/ETH/Master thesis/KalmanNet_VO


In [1]:
import os
import torch
import sys
from datetime import datetime
from Extended_sysmdl_visual import SystemModel
import numpy as np
from EKF_test_visual import EKFTest
#sys.path.insert(1, "Simulations/Pendulum/model.py")
from model import f, h_full, h_partial, h_nonlinear
import matplotlib.pyplot as plt
from KalmanNet_sysmdl import SystemModel
from KalmanNet_nn_NewArch_visual import KalmanNetNN
from Pipeline_KF_visual_old import Pipeline_KF


if torch.cuda.is_available():
    dev = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on GPU


In [2]:
NL_m = 2
NL_n = 2

m1_0 = torch.Tensor([[0.0], [0.0]]).to(dev)
m2_0 = 0 * 0 * torch.eye(NL_m).to(dev)

v = 0
q2 = 0.1
q = np.sqrt(q2)
r2 = 0.1
r = np.sqrt(r2)
h = h_full
Q = q*q*torch.eye(NL_m)
R = r*r*torch.eye(NL_n)
EPOCHS = 10
BATCH_SIZE = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4

input_ = np.load(f"Datasets/Pendulum/decimated_noisy_data/pendulum_decimated_noisy_q2_{q2:.0e}_r2_{r2:.0e}_v{v}.npz")
target_ = np.load(f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
train_input = torch.from_numpy(input_["training_set"][:, :, :]).float().to(dev)
train_target = torch.from_numpy(target_["training_set"][:, :, :]).float().to(dev)
test_input = torch.from_numpy(input_["test_set"][:, :, :]).float().to(dev)
test_target = torch.from_numpy(target_["test_set"]).float().to(dev)
cv_input = torch.from_numpy(input_["validation_set"][:200, :, :]).float().to(dev)
cv_target = torch.from_numpy(target_["validation_set"][:200, ...]).float().to(dev)

traj_length_training = train_input.shape[2]
traj_length_test = test_input.shape[2]

In [3]:
today = datetime.today()
now = datetime.now()
strToday = today.strftime("%m.%d.%y")
strNow = now.strftime("%H:%M:%S")
strTime = strToday + "_" + strNow
print("Current Time =", strTime)

system_model = SystemModel(f=f, Q=Q, h=h, R=R, T=traj_length_training, T_test=traj_length_test, prior_Q=None, prior_Sigma=None, prior_S=None)
system_model.InitSequence(m1_0, m2_0)
KNet_Pipeline = Pipeline_KF(Time=strTime, folderName="KNet", modelName="KalmanNet", data_name="Pendulum")
KNet_Pipeline.setssModel(system_model)

KNet_model = KalmanNetNN()
KNet_model.Build(system_model)
KNet_Pipeline.setModel(KNet_model)

KNet_Pipeline.setTrainingParams(fix_H_flag=1, n_Epochs=EPOCHS, n_Batch=BATCH_SIZE, learningRate=LEARNING_RATE, weightDecay=WEIGHT_DECAY)



Current Time = 05.11.22_11:57:13


In [5]:
KNet_Pipeline.NNTrain(n_Examples=train_input.shape[0], n_CV=cv_input.shape[0], model_AE=None,train_input=train_input, train_target=train_target, cv_input=cv_input, cv_target=cv_target, model_AE_conv=None, matrix_data_flag=1)

0 MSE Training : tensor(inf) [dB] MSE Validation : tensor(inf) [dB] timing  1652262949.8881977
Optimal idx: 0 Optimal : 1000 [dB]


KeyboardInterrupt: 

In [ ]:
[KNet_MSE_test_linear_arr, KNet_MSE_test_linear_avg, KNet_MSE_test_dB_avg, KNet_test]=KNet_Pipeline.NNTest(n_Test=test_input.shape[0], test_input=test_input, test_target=test_target, model_AE=None, model_AE_conv=None, matrix_data_flag=1)

In [8]:
train_input


tensor([[[-0.5888, -1.7051, -1.3227,  ..., -0.7503, -1.2675, -0.6058],
         [ 0.0096, -0.1603, -0.2370,  ..., -1.5360, -1.2074, -1.1008]],

        [[-0.9208, -0.9067, -1.2048,  ..., -0.7186, -1.2775, -0.2811],
         [-0.3302,  0.3219, -0.0843,  ..., -2.5882, -2.1960, -1.7494]],

        [[-0.4039, -0.3558,  0.1618,  ..., -0.3303, -0.6542,  0.0618],
         [ 0.4863,  0.4383, -0.1546,  ...,  0.2156,  0.1110, -0.0732]],

        ...,

        [[-0.9153, -0.7140, -0.9811,  ..., -0.0252, -1.6547, -1.1451],
         [-0.0144, -0.2078, -0.2928,  ..., -0.9667, -0.7994, -0.9113]],

        [[-0.8541, -0.6285, -1.0203,  ..., -0.8120, -0.2440, -0.2285],
         [ 0.2649,  0.2417,  0.4637,  ..., -0.3380, -1.1735, -0.7930]],

        [[ 1.1155,  1.0017,  0.9622,  ...,  0.7922,  1.0003,  1.1947],
         [ 0.1315, -0.0677, -0.1723,  ...,  1.3576,  1.3352,  1.4573]]])

0
1
2
3
4
5
6
7
8
9
